___
# Ciência dos Dados - PROJETO 2

___
## Jhonata Ferreira

## Matheus Augusto 

___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [2]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\jhona\Downloads\Ciencia dos Dados DP\Projeto 2


In [4]:
dados = pd.read_excel('spamhamV2.xlsx',sep=',')
dados.head()

,Email,Class
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [5]:
treinamento=dados.sample(frac=0.75)
treinamento.reset_index(drop = True, inplace = True)
teste=dados.drop(treinamento.index)
teste.reset_index(drop = True, inplace = True)

In [6]:
treinamento.head()

,Email,Class
0,EASTENDERS TV Quiz. What FLOWER does DOT compa...,spam
1,Tell where you reached,ham
2,Hmmm...k...but i want to change the field quic...,ham
3,How i noe... Did Ì_ specify da domain as nusst...,ham
4,Ok anyway no need to change with what you said,ham


In [7]:
teste.head()

,Email,Class
0,"swhrt how u dey,hope ur ok, tot about u 2day.l...",ham
1,"Ok da, i already planned. I wil pick you.",ham
2,Urgent! Please call 0906346330. Your ABTA comp...,spam
3,"Sorry, I'll call later in meeting",ham
4,I just really need shit before tomorrow and I ...,ham


In [8]:
Ham_t = treinamento.loc[treinamento.Class == 'ham']
Spam_t = treinamento.loc[treinamento.Class == 'spam']

In [9]:
def clean(lista):
    i=0
    while i < len (lista):
        lista[i]=lista[i].replace("_"," ")
        lista[i]=lista[i].replace("'"," ")
        lista[i]=lista[i].replace("?"," ")
        lista[i]=lista[i].replace("$"," ")
        lista[i]=lista[i].replace("*"," ")
        lista[i]=lista[i].replace("%"," ")
        lista[i]=lista[i].replace("-"," ")
        lista[i]=lista[i].replace("<"," ")
        lista[i]=lista[i].replace(">"," ")
        lista[i]=lista[i].replace("'"," ")
        lista[i]=lista[i].replace('"'," ")
        lista[i]=lista[i].replace("!"," ")
        lista[i]=lista[i].replace("."," ")
        lista[i]=lista[i].replace(","," ")
        lista[i]=lista[i].replace("|"," ")
        lista[i]=lista[i].replace(":"," ")
        lista[i]=lista[i].replace("&"," ")
        lista[i]=lista[i].replace("("," ")
        lista[i]=lista[i].replace(")"," ")
        lista[i]=lista[i].replace("+"," ")
        lista[i]=lista[i].replace("#"," ")
        lista[i]=lista[i].replace("^"," ")
        lista[i]=lista[i].replace("÷"," ")
        lista[i]=lista[i].replace("and"," ")
        lista[i]=lista[i].replace("is"," ")
        i+=1
    lista=list(filter(None, lista))

    return lista

In [48]:
#Criando listas com palavras

Ham_palavras = []
for email in Ham_t.Email:
    palavras = email.split()
    for palavra in palavras:
        Ham_palavras.append(palavra)
        
Spam_palavras = []
for email in Spam_t.Email:
    palavras = email.split()
    for palavra in palavras:
        Ham_palavras.append(palavra)

Treinamento_palavras_totais = []
for email in treinamento.Email:
    palavras = email.split()
    for palavra in palavras:
        Treinamento_palavras_totais.append(palavra)
Treinamento_palavras = np.unique(Treinamento_palavras_totais)

In [11]:
ham_clean=clean(Ham_palavras)
spam_clean=clean(Spam_palavras)
treinamento_clean=(Treinamento_palavras)


## 3. Classificador Naive-Bayes

In [12]:
#P(SPAM)
Pspam = len(Spam_t)/len(treinamento)

#P(HAM)
Pham = 1 - Pspam

#P(word/SPAM)
def PwordSPAM(word):
    word_count = Spam_palavras.count(word)
    word_count = word_count + 1
    Pdivisor = len(Spam_palavras) + len(Treinamento_palavras)
    Pwspam = word_count/Pdivisor
    return Pwspam

#P(word/HAM)
def PwordHAM(word):
    word_count = Ham_palavras.count(word)
    word_count = word_count + 1
    Pdivisor = len(Ham_palavras) + len(Treinamento_palavras)
    Pwham = word_count/Pdivisor
    return Pwham

In [13]:
# Classificador

def Class_PSPAM(mensagem):
    P = 1
    for palavra in mensagem:
        P = P * PwordSPAM(palavra)
    return P

def Class_PHAM(mensagem):
    P = 1
    for palavra in mensagem:
        P = P * PwordHAM(palavra)
    return P

In [46]:
teste['Teste'] = 0

for index in range(0,len(teste)):
    mensagem = teste.Email[index]
    mensagem = clean(mensagem.split())
    if Class_PSPAM(mensagem)*Pspam < Class_PHAM(mensagem)*Pham:
        teste.Teste.loc[index] = 'ham'
        
    else:
        teste.Teste.loc[index] = 'spam'

        

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [19]:
resultado = pd.crosstab(teste.Class, teste.Teste)
resultado

Teste,ham,spam
Class,,
ham,1188,23
spam,180,2


In [20]:
teste

,Email,Class,Teste
0,"swhrt how u dey,hope ur ok, tot about u 2day.l...",ham,ham
1,"Ok da, i already planned. I wil pick you.",ham,ham
2,Urgent! Please call 0906346330. Your ABTA comp...,spam,ham
3,"Sorry, I'll call later in meeting",ham,ham
4,I just really need shit before tomorrow and I ...,ham,ham
5,I'm good. Have you registered to vote?,ham,ham
6,"Hmm ok, i'll stay for like an hour cos my eye ...",ham,ham
7,Dear got bus directly to calicut,ham,ham
8,Mm umma ask vava also to come tell him can pla...,ham,ham
9,"Well the general price is &lt;#&gt; /oz, let ...",ham,ham


In [21]:
resultado

Teste,ham,spam
Class,,
ham,1188,23
spam,180,2


___
## 4. Curva ROC

In [45]:
Espec = []
Sens = []

for por in np.arange(0.25,1,0.01):
    teste['Teste'] = 'ham'

    for index in range(0,len(teste)):
        mensagem = teste.Email[index]
        mensagem = clean(mensagem.split())
        if Class_PSPAM(mensagem)*Pspam > por:
            teste.Teste.loc[index] = 'spam'

        
    resultado = pd.crosstab(teste.Teste, teste.Class)
    
    espec = resultado.spam[1]/(resultado.spam[0] + resultado.spam[1])
    sens = resultado.ham[0]/(resultado.ham[0] + resultado.ham[1])
    
    Espec.append(espec)
    Sens.append(sens)


IndexError: index out of bounds

In [ ]:
plt.plot(Espec,Sens, 'r')
plt.show()

In [60]:
def Pword(word):
    word_count = Treinamento_palavras_totais.count(word)
    word_count = word_count + 1
    Pdivisor = len(Treinamento_palavras_totais) + len(Treinamento_palavras)
    Pw = word_count/Pdivisor
    return Pw

def Class_P(mensagem):
    P = 1
    for palavra in mensagem:
        P = P * Pword(palavra)
    return P

In [62]:
for index in range(0,len(teste)):
    mensagem = teste.Email[index]
    mensagem = clean(mensagem.split())
    print((Class_PSPAM(mensagem)*Pspam)/Class_P(mensagem))

9.040274105162689e-07
1.296212975037099e-06
0.0030756371112780623
2.018706954081409e-05
1.950000403358454e-18
0.0002863048752584017
2.611530009043372e-07
3.5312887996828446e-06
7.359614297305451e-13
1.463069754052149e-09
0.009052973727267613
5.014060360216186e-07
7.250432196256057e-05
0.004133628499671206
4.100922386978483e-05
1.2948539103821258e-10
5.983342178679727e-19
3.8434489589126425e-19
3.2490811049808587e-16
1.043508090648484e-12
1.4229320323718372e-07
5.264065326632009e-09
5.7246480783673506e-33
2.8258765576628743e-08
4.0330255766153186e-05
208.65893323078035
0.0005356783891191282
0.0057788476362543575
1.2344689928903326e-15
2.7067500218674758e-08
6.205312885638529e-05
4.7048449288538445e-06
0.0008049852585175896
0.0021959673768163876
2.7112166760993654e-12
4.324978986623583e-06
5.434538795103113
3.426544798684493e-10
1.220242812667388e-07
8.56488603245936e-12
1.9941905765150885e-14
5.338592673449095e-30
1.497633266344679e-12
8.956556524067324e-06
17.594460299469116
3.81249519

ZeroDivisionError: float division by zero

___
## 5. Qualidade do Classificador alterando a base de treinamento